__Toutes les fonctions définies dans le notebook pre_processing sont mises dans un fichier Python qu'on importe__

In [1]:
import os
import dicom
import pylab
import os
import numpy as np
import matplotlib.pyplot as plt

from scipy import ndimage as ndi
from skimage.filters import roberts, sobel
from skimage.segmentation import clear_border
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing

import pre_processing as pp

In [3]:
INPUT_FOLDER = '/home/lucsoret/Projet/Supelec/CEI/Data/sample_images'
patients = os.listdir(INPUT_FOLDER)
patients.sort()

In [8]:
first_patient_path = os.path.join(INPUT_FOLDER,patients[0])
first_patient = pp.load_scan(first_patient_path)
total_images = os.listdir(first_patient_path)


first_patient_image =  pp.PixelData_to_HU(first_patient)
image =  np.stack([pp.center(pp.normalize(pp.get_segmented_lungs(s))) for s in first_patient_image])
image,spacing = pp.resample(image,first_patient)

Premier problème : Nombre de slices différentes d'un patient à l'autre (meme après resample) :

In [11]:
L = len(patients)
for i in range(L):
    first_patient_path = os.path.join(INPUT_FOLDER,patients[i])
    first_patient = pp.load_scan(first_patient_path)
    first_patient_image = pp.PixelData_to_HU(first_patient)
    image = np.stack(s for s in first_patient_image)
    image,spacing = pp.resample(image,first_patient)
    print(image.shape)


(335, 306, 306)
(320, 347, 347)
(332, 340, 340)
(275, 320, 320)
(365, 279, 279)
(353, 360, 360)
(350, 340, 340)
(308, 355, 355)
(328, 360, 360)
(305, 390, 390)
(272, 330, 330)
(360, 350, 350)
(398, 309, 309)
(294, 300, 300)
(304, 424, 424)
(366, 370, 370)
(315, 310, 310)
(354, 259, 259)
(342, 392, 392)
(282, 308, 308)


L'avantage de notre re-sampling, c'est qu'on a des images homogènes en distance réel d'un scan à l'autre. De plus, cela permet de "condenser" un peu nos tenseurs qui ont des tailles assez semblables après resampling. La méthode la plus basique alors va être d'effectuer un __zero_pading__, pour avoir des données de même taille 

In [59]:
def zero_padding(im,max_z=400,max_x=450,max_y=450):
    #Les dimensions sont prises empiriquement, au vu des shape montrees precedemment 
    borders = (max_z,max_x,max_y)
    fill_total = [[0,0],[0,0],[0,0]]
    for i in range(3):
        current_shape = im.shape[i]
        if (current_shape < borders[i]):
            fill = borders[i] - current_shape
            fill_left = fill/2
            fill_right = fill/2 + fill % 2
            fill_total[i] = (fill_left,fill_right)
    im = np.pad(im,((fill_total[0][0],fill_total[0][1]),(fill_total[1][0],fill_total[1][1]),(fill_total[2][0],fill_total[2][1])),'constant',constant_values=0)
    #Si jamais notre image depasse les dimensions du zero padding, on va la cut de base
    im = im[0:max_z,0:max_x,0:max_y]
    return im

In [63]:
i = 10
first_patient_path = os.path.join(INPUT_FOLDER,patients[i])
first_patient = pp.load_scan(first_patient_path)
first_patient_image = pp.PixelData_to_HU(first_patient)
image = np.stack(s for s in first_patient_image)
image,spacing = pp.resample(image,first_patient)
zero_padding(image)